In [2]:
from sklearnex import patch_sklearn
patch_sklearn()
from sklearn.neighbors import NearestNeighbors
import lightgbm as lightgbm
import numpy as np
import pandas as pd
import pickle



Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [3]:
with open('/data/Python Projects/KNN-LETOR/valid_svm_features_avg.pkl', 'rb') as f:
    preprocessed_train_cum_features = pickle.load(f) #main dict train with qids and features

In [4]:
with open('/data/Python Projects/KNN-LETOR/test_svm_features_avg.pkl', 'rb') as f:
    preprocessed_test_cum_features = pickle.load(f) #main dict with test qids and features

In [5]:
with open('/data/Python Projects/KNN-LETOR/valid_svm_features_rel.pkl', 'rb') as f:
    train_data_all = pickle.load(f)

In [6]:
with open('/data/Python Projects/KNN-LETOR/test_svm_features_rel.pkl', 'rb') as f:
    test_data_all = pickle.load(f)

In [7]:
preprocessed_train_cum_features= pd.DataFrame(preprocessed_train_cum_features)
preprocessed_test_cum_features= pd.DataFrame(preprocessed_test_cum_features)
train_data_all= pd.DataFrame(train_data_all)
test_data_all= pd.DataFrame(test_data_all)

In [8]:
qid_avg_train = preprocessed_train_cum_features["q_id"].tolist()
features_avg_train = preprocessed_train_cum_features["features"].tolist()
qid_avg_train = np.vstack(qid_avg_train)
features_avg_train = np.vstack(features_avg_train)

qid_avg_test = preprocessed_test_cum_features["q_id"].tolist()
features_avg_test = preprocessed_test_cum_features["features"].tolist()
qid_avg_test = np.vstack(qid_avg_test)
features_avg_test = np.vstack(features_avg_test)

qid_all_train= train_data_all["q_id"].tolist()
relevance_all_train = train_data_all["relevance"].tolist()
features_all_train = train_data_all["features"].tolist()
qid_all_train = np.vstack(qid_all_train)
relevance_all_train = np.vstack(relevance_all_train)
features_all_train = np.vstack(features_all_train)

qid_all_test = test_data_all["q_id"].tolist()
relevance_all_test = test_data_all["relevance"].tolist()
features_all_test = test_data_all["features"].tolist()

qid_all_test = np.vstack(qid_all_test)
relevance_all_test = np.vstack(relevance_all_test)
features_all_test = np.vstack(features_all_test)


In [40]:

import time
#find the nearest neighbour
#key2: features
#key1: qids
def get_nearest_neighbour(test_q, features_avg_train,k):
    knn = NearestNeighbors(n_neighbors=k)
    features_avg_train = np.array(features_avg_train)
    
    knn.fit(features_avg_train)
    test_q = test_q.reshape((1, -1))
    dist_loc = knn.kneighbors(test_q)
    dist = dist_loc[0]
    loc = dist_loc[1]
    return dist, loc
#these have relevance score, qids, and features
#key_0 = relevance score, key_qids, key2_features list
"""
test_unique_qids = []    # a list for getting the qids of tests
for d in qid_avg_test:
    if d not in test_unique_qids:
        test_unique_qids.append(d)
"""
k= 300
#qids_test = []
train_ids_group =[]
#len(qid_all_train)
for i in range(0,5):
    print(i)
    #get features corresponding to the qids of the test
    #qids_test.append(qid)
    #test_q = #[d['features'] for d in preprocessed_train_cum_features if d.get('q_id') == key1_value]
    #get the nearest neighbour of the test_q
    train_feature_single = features_avg_train[i]
    
    #features_avg_test = features_avg_test.reshape((features_avg_test.shape[0],-1))
    
    features_avg_train = features_avg_train.reshape((features_avg_train.shape[0], 220))
    
    dist , loc = get_nearest_neighbour(train_feature_single, features_avg_train,k)
    #get the values of the features of the trainset
    selected_train_set = features_avg_train[loc]
    #get the nearest query ids and add them to this list
    matching_train_qids = qid_avg_train[loc]
    matching_train_qids = matching_train_qids.reshape(-1,1)
    new_train_features = []
    new_train_qids = []
    new_train_relevances = []
    for qid in matching_train_qids:
        train_data_ids = np.where(qid_all_train == qid)[0]
        for k in range(0, len(train_data_ids)):
            new_train_qids.append(qid_all_train[train_data_ids[k]])
            new_train_features.append(features_all_train[train_data_ids[k]])
            new_train_relevances.append(relevance_all_train[train_data_ids[k]])
    
    new_train_relevances = np.array(new_train_relevances)
    new_train_features = np.array(new_train_features)
    new_train_qids = np.array(new_train_qids)
    
    #add k-neareset ids to a list of ids 
    train_ids_group.append(matching_train_qids.flatten())
    # Reshape "relevance" and "qid" arrays
    new_train_relevances = new_train_relevances.reshape(-1)
    new_train_qids = new_train_qids.reshape(-1)
    #new_train_features = new_train_features.tolist()
    # Concatenate arrays and list
    #train_data = np.column_stack((new_train_relevances, new_train_qids, new_train_features))
    data_train_tmp = {
    'relevance': new_train_relevances,
    'qid': new_train_qids,
    'features': new_train_features.tolist()  # Convert features to a list of lists
    }

    # Create a DataFrame
    train_group = pd.DataFrame(data_train_tmp)
    
    qids_train = train_group.groupby("qid")["qid"].count().to_numpy()
    #qids_test = test_group.groupby("qid")["qid"].count().to_numpy()
    
    new_train_relevances = new_train_relevances.astype(int)
    #new_test_relevances = new_test_relevances.astype(int)
    a = time.time()
    ranker = lightgbm.LGBMRanker(
                        objective="lambdarank",
                        boosting_type = "gbdt",
                        n_estimators = 1,
                        importance_type = "gain",
                        force_col_wise=True,
                        metric= "ndcg",
                        num_leaves = 250,
                        learning_rate = 0.05,
                        max_depth = -1,
                        device ='gpu',
                        label_gain =[i for i in range((new_train_relevances.max()) + 1)])
    
    # Training the model
    ranker.fit(
          X=new_train_features,
          y=new_train_relevances,
          group= qids_train)
          #eval_set=[(new_train_features, new_train_relevances)]
          #eval_group=qids_train,
          #eval_at=[5, 10])
          #eval_set=[(new_train_features, new_train_relevances),(new_test_features, new_test_relevances)],
          #eval_group=[qids_train, qids_test],
          #eval_at=[5, 10])
    ranker.booster_.save_model('mode'+str(i)+'.txt')
    b = time.time()
    print(b-a)

0
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 19928
[LightGBM] [Info] Number of data points in the train set: 81601, number of used features: 194
[LightGBM] [Info] Using GPU Device: NVIDIA RTX A5000, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 93 dense feature groups (7.47 MB) transferred to GPU in 0.010575 secs. 1 sparse feature groups
0.8528599739074707
1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 20154
[LightGBM] [Info] Number of data points in the train set: 82847, number of used features: 193
[LightGBM] [Info] Using GPU Device: NVIDIA RTX A5000, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 94 dense feature groups (7.5

In [1]:
from sklearn.metrics import ndcg_score
k = 1
for i in range(0, len(features_avg_test)):
    test_feature = features_avg_test[i]
    
    features_avg_test = features_avg_test.reshape((features_avg_test.shape[0],-1))
    
    features_avg_train = features_avg_train.reshape((features_avg_train.shape[0], 220))
    
    dist , loc = get_nearest_neighbour(test_feature, features_avg_train,k)
    #get the values of the second key(features) of the trainset
    selected_train_set = features_avg_train[loc]
    #get the nearest query ids and add them to this list
    matching_train_qids = qid_avg_train[loc]
    matching_train_qids = matching_train_qids.reshape(-1,1)
    print(train_ids_group)
    asdsad
    max_common = 0
    group_id = 0
    for j in range(0, len(train_ids_group)):
        
        commons_len = len(np.intersect1d(train_ids_group[j], matching_train_qids))
        if(commons_len > max_common):
            max_common = commons_len
            group_id = j
    
    bst = lightgbm.Booster(model_file='mode'+str(group_id)+'.txt')
    
    test_data_ids = np.where(qid_all_test == qid_avg_test[i])[0]
    new_test_qids = []
    new_test_features = []
    new_test_relevances = []
    for k in range(0, len(test_data_ids)):
            new_test_qids.append(qid_all_test[test_data_ids[k]])
            new_test_features.append(features_all_test[test_data_ids[k]])
            new_test_relevances.append(relevance_all_test[test_data_ids[k]])
            
    new_test_qids = np.array(new_test_qids)
    new_test_features = np.array(new_test_features)
    new_test_relevances = np.array(new_test_relevances)
    
    new_test_relevances = new_test_relevances.flatten() 
   
    test_pred = bst.predict(new_test_features)
    new_test_relevances = new_test_relevances.reshape((1, -1))
    test_pred = test_pred.reshape((1, -1))
    #print(test_pred)
    print(ndcg_score(new_test_relevances, test_pred))

NameError: name 'features_avg_test' is not defined

In [65]:
#print((new_train_relevances[0:10]))

#print(test_data_all)
#print(test_data_all[test_data_all['q_id']==263])


     q_id  relevance                                           features
0     263        0.0  [1.0, 1444.0, 2.0, 10.0, 53.0, 120.0, 3.0, 556...
1     263        0.0  [1000000.0, 1004.0, 0.0, 0.0, 53.0, 10.0, 1.0,...
2     263        0.0  [1000000.0, 1004.0, 0.0, 0.0, 53.0, 130.0, 1.0...
3     263        0.0  [1000000.0, 1011.0, 0.0, 0.0, 53.0, 106.0, 1.0...
4     263        0.0  [1000000.0, 1017.0, 0.0, 1.0, 53.0, 50.0, 1.0,...
..    ...        ...                                                ...
994   263        0.0  [9589.0, 984.0, 2.0, 63.0, 53.0, 130.0, 2.0, 5...
995   263        0.0  [965084.0, 727.0, 0.0, 0.0, 53.0, 99.0, 1.0, 5...
996   263        0.0  [967780.0, 1144.0, 0.0, 7.0, 53.0, 130.0, 2.0,...
997   263        0.0  [993637.0, 899.0, 2.0, 5.0, 53.0, 100.0, 1.0, ...
998   263        0.0  [996789.0, 626.0, 0.0, 0.0, 53.0, 95.0, 1.0, 5...

[999 rows x 3 columns]


In [94]:
ranker.evals_result_

{'training': OrderedDict([('ndcg@5', [0.7490828397399896]),
              ('ndcg@10', [0.7920798893578783])]),
 'valid_1': OrderedDict([('ndcg@5', [0.8687949224876582]),
              ('ndcg@10', [0.7751482523375255])])}